In [23]:
## Regression problem statement -- output is salary of employees (continuous values)

import pickle
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder,OneHotEncoder

In [24]:
data=pd.read_csv('Churn_Modelling.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [25]:
data=data.drop(['RowNumber','CustomerId','Surname'],axis=1)

In [26]:
## Encode categorical variable
label_encoder_gender=LabelEncoder()
data['Gender']=label_encoder_gender.fit_transform(data['Gender'])
data  
## Female-0, Male-1

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,1,39,5,0.00,2,1,0,96270.64,0
9996,516,France,1,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,0,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,1,42,3,75075.31,2,1,0,92888.52,1


In [27]:
##Onehot encode 'Geography' 
##Can't use LabelEncoder because the categories aren't binary, there are more than 2 categories, so 0,1,2 so the model will interpret it as Germany>France>Spain, which is an absurd assumption, hence here we need one hot encoding

from sklearn.preprocessing import OneHotEncoder
onehotgeo=OneHotEncoder(sparse_output=False) ## Won't give sparse matrix -- matrix with mostly zeros and few non-zeros, sparse matrix stores the positions of non-zero values onl to save memory
geoenc=onehotgeo.fit_transform(data[['Geography']]) ## Always double bracket as 2D matrix mandatory
col=onehotgeo.get_feature_names_out()
geo_df=pd.DataFrame(geoenc,columns=col)
geo_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [28]:
## Combine one hot encoded columsn with the original data

data=pd.concat([data.drop('Geography',axis=1),geo_df],axis=1)


In [29]:
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [30]:
# ✅ Import necessary libraries
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# ✅ Split dataset into independent (X) and dependent (Y) features
X = data.drop('EstimatedSalary', axis=1)  # Features used to predict (e.g., age, salary, credit score)
Y = data['EstimatedSalary']               # Target variable (whether the customer exited or not)

# ✅ Split the data into training and testing sets
# test_size=0.2 → 20% of data will be used for testing
# random_state=42 → sets a seed so the same random split happens every time (for reproducibility)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# ✅ Feature Scaling
# ⚠️ Why scale? Many ML models are sensitive to the scale of features.
#    For example, 'Age' might range from 18 to 100, while 'Balance' might go up to 1,000,000.
#    Models like logistic regression, SVM, neural networks, and KNN can give undue importance
#    to features with larger numerical values.

# 🎯 Intuition:
#    Imagine a race between a swimmer, runner, and cyclist—but you measure them in different units (laps, km, pedal strokes).
#    It's unfair unless you bring them all to the same unit.
#    Similarly, we scale all features to the same standard scale (mean = 0, std = 1) so the model can treat them equally.

# ✅ StandardScaler standardizes each feature by removing the mean and scaling to unit variance
scaler = StandardScaler()

# Fit the scaler on training data and transform it
X_train = scaler.fit_transform(X_train)

# Use the same scaler (trained on training data) to transform test data
X_test = scaler.transform(X_test)
X_train.shape ## (a,b) -- a is number of rows and b is number of input features
# ⚠️ Note: Never fit the scaler on test data — this avoids data leakage.


(8000, 12)

In [31]:
import pickle
# Load the saved LabelEncoder for 'Gender'
with open('label_encoder_gender.pkl', 'rb') as file:
    label_encoder_gender = pickle.load(file)

# Load the saved OneHotEncoder for 'Geography'
with open('onehot_encoder_geo.pkl', 'rb') as file:
    onehotgeo = pickle.load(file)

with open('scalerreg.pkl','wb') as file:
    pickle.dump(scaler,file)

In [32]:
## ANN Regression problem statement
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [33]:
model=Sequential([
    Dense(64,activation='relu',input_shape=(X_train.shape[1],)),    ## First hidden layer connected to input
    Dense(32,activation='relu'),  ## Second hidden layer
    Dense(1)  ## Output layer for regression
])

## compile the model
model.compile(optimizer='adam',loss='mean_absolute_error',metrics=['mae'])

In [34]:

opt=tf.keras.optimizers.Adam(learning_rate=0.01) ## many optimizers are there apart from Adam
lo=tf.keras.losses.MeanAbsoluteError()

In [35]:
model.compile(optimizer=opt,loss=lo,metrics=['mae'])

In [36]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 64)                832       
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dense_5 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [37]:
"""
📌 SETTING UP TENSORBOARD IN KERAS / TENSORFLOW — FULL INTUITION WITH CODE AND COMMENTS

-------------------------------------------------------
🔹 PURPOSE:
TensorBoard is a visualization tool that lets us monitor the training of our model.
It provides graphs for:
- Loss & accuracy over epochs
- Histograms of weights and biases
- Model graph structure
- Learning rate trends
-------------------------------------------------------
"""

# 🔸 STEP 1: Import necessary modules
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping
import datetime

"""
We import:
- TensorBoard: for logging training stats
- EarlyStopping: (optional) stops training early if no improvement
- datetime: to create unique folder names based on current date & time
"""

# 🔸 STEP 2: Create a unique log directory for TensorBoard using datetime
log_dir = "regressionlogs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

"""
📍 Intuition Behind datetime:
- datetime.datetime.now() gives the current date and time.
- strftime("%Y%m%d-%H%M%S") formats it as:
    %Y → Year   (e.g., 2025)
    %m → Month  (e.g., 08)
    %d → Day    (e.g., 03)
    %H → Hour   (24hr format)
    %M → Minute
    %S → Second
- So, datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
  might return something like "20250803-225612"
- This makes your log folder: logs/fit20250803-225612
- Each training run has a unique folder → logs don’t overwrite
"""

# 🔸 STEP 3: Initialize TensorBoard callback
tensorboard_callback = TensorBoard(
    log_dir=log_dir,        # where logs will be stored
    histogram_freq=1,       # log histograms every epoch
    write_graph=True,       # log the computational graph
    write_images=False      # optionally log model weights as images
)

In [38]:
"""
📌 EARLY STOPPING IN KERAS / TENSORFLOW — FULL EXPLANATION WITH COMMENTS & GENERAL SYNTAX

🔹 WHAT IS EARLY STOPPING?
EarlyStopping is a built-in Keras callback that:
→ Monitors model performance during training
→ Stops training when it detects no further improvement
→ Optionally restores the model weights from the best epoch

This is useful for:
✅ Preventing overfitting
✅ Saving time and compute
✅ Automatically picking the best version of the model

-------------------------------------------------------
🔹 GENERAL SYNTAX:

EarlyStopping(
    monitor='val_loss',          # Metric to monitor ('val_loss', 'val_accuracy', etc.)
    patience=5,                  # Number of epochs to wait before stopping if no improvement
    restore_best_weights=True,  # Restore model weights from the best epoch
    min_delta=0,                # Minimum change to qualify as an improvement (optional)
    mode='auto',                # 'auto', 'min', or 'max' — auto figures out based on monitored metric
    verbose=1                   # Show logs when training stops (optional)
)

-------------------------------------------------------
🔹 EXAMPLE SETUP:
"""

# 🔸 Import EarlyStopping
from tensorflow.keras.callbacks import EarlyStopping

# 🔸 Define the callback
early_stopping_callback = EarlyStopping(
    monitor='val_loss',          # Stop training when validation loss stops improving
    patience=10,                  # Wait 5 epochs for improvement
    restore_best_weights=True,  # Revert to best model after stopping
    verbose=1                   # Print message when training is stopped
)

"""
-------------------------------------------------------
🔹 USAGE IN TRAINING:
Just pass it as a callback to model.fit()

model.fit(
    X_train, Y_train,
    epochs=100,
    validation_data=(X_test, Y_test),
    callbacks=[early_stopping_callback]
)

-------------------------------------------------------
🔸 MONITOR OPTIONS:
- 'val_loss': Most common, looks at validation loss
- 'val_accuracy': For classification tasks

🔸 PATIENCE:
- Number of epochs to allow for no improvement
- E.g., if patience=5, training stops if no better result in 5 rounds

🔸 RESTORE_BEST_WEIGHTS:
- True = after stopping, model weights will revert to the best epoch
- False = model keeps the weights from the last epoch (not recommended)
"""


"\n-------------------------------------------------------\n🔹 USAGE IN TRAINING:\nJust pass it as a callback to model.fit()\n\nmodel.fit(\n    X_train, Y_train,\n    epochs=100,\n    validation_data=(X_test, Y_test),\n    callbacks=[early_stopping_callback]\n)\n\n-------------------------------------------------------\n🔸 MONITOR OPTIONS:\n- 'val_loss': Most common, looks at validation loss\n- 'val_accuracy': For classification tasks\n\n🔸 PATIENCE:\n- Number of epochs to allow for no improvement\n- E.g., if patience=5, training stops if no better result in 5 rounds\n\n🔸 RESTORE_BEST_WEIGHTS:\n- True = after stopping, model weights will revert to the best epoch\n- False = model keeps the weights from the last epoch (not recommended)\n"

In [39]:
history=model.fit(
    X_train, Y_train,
    epochs=100,
    validation_data=(X_test, Y_test),
    callbacks=[tensorboard_callback,early_stopping_callback]
)

Epoch 1/100
250/250 [==============================] - 1s 2ms/step - loss: 84359.7812 - mae: 84359.7812 - val_loss: 52671.8906 - val_mae: 52671.8906
Epoch 2/100
250/250 [==============================] - 0s 1ms/step - loss: 50441.2227 - mae: 50441.2227 - val_loss: 50577.3711 - val_mae: 50577.3711
Epoch 3/100
250/250 [==============================] - 0s 1ms/step - loss: 49933.9844 - mae: 49933.9844 - val_loss: 50416.0586 - val_mae: 50416.0586
Epoch 4/100
250/250 [==============================] - 0s 1ms/step - loss: 49774.1914 - mae: 49774.1914 - val_loss: 50367.0547 - val_mae: 50367.0547
Epoch 5/100
250/250 [==============================] - 0s 1ms/step - loss: 49680.1836 - mae: 49680.1836 - val_loss: 50319.6367 - val_mae: 50319.6367
Epoch 6/100
250/250 [==============================] - 0s 1ms/step - loss: 49622.7773 - mae: 49622.7773 - val_loss: 50285.7656 - val_mae: 50285.7656
Epoch 7/100
250/250 [==============================] - 0s 1ms/step - loss: 49546.6367 - mae: 49546.6367 - 

In [40]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [41]:
%tensorboard --logdir regressionlogs/fit

Reusing TensorBoard on port 6006 (pid 16752), started 0:51:07 ago. (Use '!kill 16752' to kill it.)

In [42]:
## Evaluate model based on test data
test_loss,test_mae=model.evaluate(X_train,Y_train)
print(f"Train MAE: {test_mae}")
print(f"Train Loss: {test_loss}")

250/250 [==============================] - 0s 717us/step - loss: 49469.1719 - mae: 49469.1719
Train MAE: 49469.171875
Train Loss: 49469.171875


In [43]:
## Evaluate model based on test data
test_loss,test_mae=model.evaluate(X_test,Y_test)
print(f"Train MAE: {test_mae}")
print(f"Train Loss: {test_loss}")

63/63 [==============================] - 0s 737us/step - loss: 50285.7656 - mae: 50285.7656
Train MAE: 50285.765625
Train Loss: 50285.765625


In [44]:
model.save('regression_model.h5')

c:\Users\Harjeet Singh Pannu\Desktop\ANN project\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
